# 0.1 Import packages and data 

In [1]:
# Packages for data manipulation
import numpy as np
import pandas as pd
import json

In [2]:
# Packages for dataviz
import plotly.express as px
import plotly.tools as tls
import cufflinks as cf
import matplotlib.pyplot as plt

In [3]:
# configuration for plotly
template = "plotly_dark"

# offline configuration of cufflinks
cf.go_offline()

In [4]:
# Load the data
data = pd.read_csv('../data/raw.csv')

In [5]:
pd.set_option('display.max_columns', 110)

## Pieces of documentation about the variables
* https://rstudio-pubs-static.s3.amazonaws.com/365075_ec9ebe4da4cc465ba9beaef25cda6bad.html
* https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data

# 0.2 Quick look at the dataset

In [6]:
data.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,958,https://www.airbnb.com/rooms/958,2.019100e+13,10/14/19,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under const...,"Newly remodeled, modern, and bright garden uni...",New update: the house next door is under const...,none,*Quiet cul de sac in friendly neighborhood *St...,Due to the fact that we have children and a do...,*Public Transportation is 1/2 block away. *Ce...,*Full access to patio and backyard (shared wit...,A family of 4 lives upstairs with their dog. N...,* No Pets - even visiting guests for a short t...,NaN,NaN,https://a0.muscache.com/im/pictures/b7c2a199-4...,NaN,1169,https://www.airbnb.com/users/show/1169,Holly,7/31/08,"San Francisco, California, United States",We are a family with 2 boys born in 2009 and 2...,within a day,100%,NaN,t,https://a0.muscache.com/im/pictures/efdad96a-3...,https://a0.muscache.com/im/pictures/efdad96a-3...,Duboce Triangle,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,"San Francisco, CA, United States",Duboce Triangle,Western Addition,NaN,San Francisco,CA,94117,San Francisco,"San Francisco, CA",US,United States,37.76931,-122.43386,t,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Pets liv...",NaN,$170.00,"$1,120.00","$4,200.00",$100.00,$100.00,2,$25.00,1,30,1,1,30,30,1.0,30.0,2 months ago,t,3,8,16,85,10/14/19,217,52,7/23/09,9/21/19,97.0,10.0,10.0,10.0,10.0,10.0,9.0,t,STR-0001256,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1,0,0,1.74
1,3850,https://www.airbnb.com/rooms/3850,2.019100e+13,10/14/19,Charming room for two,Your own private room plus access to a shared ...,This room can fit two people. Nobody else will...,Your own private room plus access to a shared ...,none,"This is a quiet, safe neighborhood on a substa...",House Rule footnotes: 1.\tI don’t allow check ...,Public transit service to my house is outstand...,"During the hours of 7:00 to 22:00, your access...","I live at the house in my own bedroom, and I w...",Airbnb has no default house rules that hosts c...,NaN,NaN,https://a0.muscache.com/im/pictures/b6e97d2a-1...,NaN,4921,https://www.airbnb.com/users/show/4921,Kevin,12/8/08,"San Francisco, California, United States",I am a retired software developer now raising ...,within an hour,100%

In [7]:
pd.options.display.max_rows = 999

data.host_neighbourhood.value_counts()

Mission District                   592
SoMa                               555
Potrero Hill                       426
Noe Valley                         418
Richmond District                  351
The Castro                         334
Bernal Heights                     333
Western Addition/NOPA              313
Nob Hill                           286
Outer Sunset                       284
Pacific Heights                    173
Bayview                            171
Cambridge                          170
Haight-Ashbury                     167
Union Square                       163
Downtown                           163
Chinatown                          142
Lower Haight                       141
Excelsior                          124
Telegraph Hill                     122
Inner Sunset                       117
Marina                             100
Duboce Triangle                     96
Sunnyside                           92
Parkside                            88
Alamo Square             

In [8]:
#data.columns
data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       ...
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object', length=106)

In [9]:
#data type
data.dtypes.value_counts()

object     62
float64    24
int64      20
dtype: int64

To simplify this notebook, the cleanning of the variables is organized as followed:
* Quantitative
* Categorical 
* Boolean
* Text
* Spatial
* To drop
* I don't know yet

In [10]:
# Outlier detection
#col_to_drop = []
row_to_drop = []

In [11]:
# control objects
var_quant = []
var_cat = []
var_bool = []
var_text = []
var_date = []
var_space = []
var_to_drop = []
var_TODO = []
var_cible = []

# 1. Y variable

In [12]:
var_cible.append("review_scores_rating")

On suppose qu'on ne dispose pas des autres *rating* pour expliquer Y

In [13]:
var_to_drop.extend( ['review_scores_checkin', 'review_scores_cleanliness', 'review_scores_communication',
                     'review_scores_location', 'review_scores_accuracy', 'review_scores_value', 'reviews_per_month'] )

# 2. Data cleaning

# 2.1 Variables with no cleaning needed

There is no early remarks regarding theses variables. Nothing special came out when looking at them.

In [14]:
var_quant.extend(["accommodates", "guests_included"])

In [15]:
var_text.extend(['name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes',
                'transit', 'access', 'interaction', 'house_rules', 'host_about'])

In [16]:
var_space.extend(['latitude', 'longitude'])

Nous n'utilisons pas le zipcode dans notre traitement des variables categoriques

In [17]:
var_to_drop.append("zipcode")

In [18]:
data[var_text]

,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,host_about
0,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under const...,"Newly remodeled, modern, and bright garden uni...",New update: the house next door is under const...,*Quiet cul de sac in friendly neighborhood *St...,Due to the fact that we have children and a do...,*Public Transportation is 1/2 block away. *Ce...,*Full access to patio and backyard (shared wit...,A family of 4 lives upstairs with their dog. N...,* No Pets - even visiting guests for a short t...,We are a family with 2 boys born in 2009 and 2...
1,Charming room for two,Your own private room plus access to a shared ...,This room can fit two people. Nobody else will...,Your own private room plus access to a shared ...,"This is a quiet, safe neighborhood on a substa...",House Rule footnotes: 1.\tI don’t allow check ...,Public transit service to my house is outstand...,"During the hours of 7:00 to 22:00, your access...","I live at the house in my own bedroom, and I w...",Airbnb has no default house rules that hosts c...,I am a retired software developer now raising ...
2,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,I love how our neighborhood feels quiet but is...,All the furniture in the house was handmade so...,The train is two blocks away and you can stop ...,"Our deck, garden, gourmet kitchen and extensiv...",NaN,"Please respect the house, the art work, the fu...",Philip: English transplant to the Bay Area and...
3,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,"Settle down, S.F. resident, student, hospital,...",Nice and good public transportation. 7 minute...,"Shopping old town, restaurants, McDonald, Whol...",Wi-Fi signal in common areas. Large eat in k...,N Juda Muni and bus stop. Street parking.,NaN,NaN,"No party, No smoking, not for any kinds of smo...",7 minutes walk to UCSF hospital & school campu...
4,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,"Settle down, S.F. resident, student, hospital,...",Nice and good public transportation. 7 minute...,NaN,Wi-Fi signal in common areas. Large eat in k...,"N Juda Muni, Bus and UCSF Shuttle. small shopp...",NaN,NaN,no pet no smoke no party inside the building,7 minutes walk to UCSF hospital & school campu...
5,Historic Alamo Square Victorian,Pls email before booking. Interior featured i...,Please send us a quick message before booking ...,Pls email before booking. Interior featured i...,NaN,tax ID on file tax ID on file,NaN,Guests have access to everything listed and sh...,NaN,House Manual and House Rules will be provided ...,I'm an Interior Stylist living in SF. \r\n\r\n...
6,"Mission Sunshine, with Private Bath","Welcome to ""The Mission,"" the sunniest neighbo...","Your sunny room looks out over a lush garden, ...","Welcome to ""The Mission,"" the sunniest neighbo...",Located between Valencia Street and Dolores Pa...,"We live in a dense, urban neighborhood, and ou...",It's an easy and enjoyable 10 minute stroll to...,You'll have your own bedroom and private bathr...,"We are experienced Airbnb hosts, and have had ...","PLEASE READ: By booking, you agree to all of t...",Ivan is a publisher of a book for 4 year olds....
7,Artful Potrero Separate Floor with Garden,A Unique Guest Suite! A Spacious Art Filled ...,"It is unique, very spacious [800 sq. ft. with...",A Unique Guest Suite! A Spacious Art Filled ...,"It is a distinct neighborhood, with easy acces...",NaN,There are buses within a block to connect to a...,"Laundry privileges, Spacious Garden with City ...",We are available to make suggestions to enhanc...,We are respectful of the privacy of our renter...,The apartment is on the ground floor of the ho...
8,Victorian Suite in Inner Mission,NaN,Please read this before you book! Second floor...,Please read 

# 2.2 Variables to drop

## *id*

In [19]:
data['id'].nunique()

8111

In [20]:
var_to_drop.append('id')

## 2.2.1 Empty columns

All these columns are filled with NAN so they are removed

In [21]:
empty_l = ["thumbnail_url", "medium_url", "xl_picture_url",
                    "host_acceptance_rate", "neighbourhood_group_cleansed"]

In [22]:
def print_empty(data, l):
    for col in l:
        print(data[col].isna().sum())
        print("")

In [23]:
#print_empty(data, empty_l)

In [24]:
var_to_drop.extend(empty_l)

In [25]:
#print((data['experiences_offered'] == "none").sum())

In [26]:
var_to_drop.append('experiences_offered')

## 2.2.2 Variables with one value

The variable is a constant so this won't be useful.

In [27]:
one_l = ['state', 'country_code', 'country', 'has_availability',
            'jurisdiction_names','is_business_travel_ready', 'street']

In [28]:
def print_unique(data, l):
    for col in l:
        print(data[col].value_counts())
        print("")

In [29]:
#print_unique(data, one_l)

In [30]:
var_to_drop.extend(one_l)

## 2.2.3 Useless variables

*scrape_id*, *calendar_last_scraped* and *last_scraped* are not useful in this study. We are not interested in the data scrapping.

In [31]:
var_to_drop.extend(["scrape_id", "last_scraped", "calendar_last_scraped"])

The content of theses variables seems unclear and unecessary
* *minimum_minimum_nights*
* *maximum_minimum_nights*
* *minimum_maximum_nights*
* *maximum_maximum_nights*  

In [32]:
var_to_drop.extend(['minimum_minimum_nights', 'maximum_minimum_nights', 
                    'minimum_maximum_nights', 'maximum_maximum_nights'])

Dans cette étude, nous allons aussi supprimer les variables contenant seulement des *url*.

In [33]:
var_to_drop.extend(['listing_url', 'picture_url', 'host_url',
                    'host_thumbnail_url', 'host_picture_url'])

Le nom d'un hote ne parait pas pertinent dans cette étude.

In [34]:
var_to_drop.extend(['host_name', 'host_id'])

In [35]:
var_to_drop.append('calendar_updated')

La colonne Market contient 2 valeurs differentes. La valeur "D.C." n'a pas l'air abberante, nous allons donc garder cette ligne et enlever la colonne.

In [36]:
data['market'].value_counts()

San Francisco    8089
D.C.                1
Name: market, dtype: int64

In [37]:
var_to_drop.append('market')

In [38]:
data[data['market'] == "D.C."]

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
3088,16241036,https://www.airbnb.com/rooms/16241036,2.019100e+13,10/14/19,Cozy Noe Valley Getaway,A sunny 1 bedroom in the heart of beautiful No...,"1 bedroom apartment with full kitchen, shower,...",A sunny 1 bedroom in the heart of beautiful No...,none,"Noe Valley neighborhood, J Church st. Muni sto...",NaN,Walk to many neighborhood shops and restaurant...,"wifi, kitchen, laundry room","text messaging is preferred, or a call if nece...",*****CHECKIN***Please contact me regarding lat...,NaN,NaN,https://a0.muscache.com/im/pictures/5c2073a6-0...,NaN,6884880,https://www.airbnb.com/users/show/6884880,Mary,6/12/13,"San Francisco, California, United States",NaN,within a few hours,100%,NaN,f,https://a0.muscache.com/im/pictures/aa06bacb-8...,https://a0.muscache.com/im/pictures/aa06bacb-8...,Noe Valley,1.0,1.0,"['email', 'phone', 'reviews', 'offline_governm...",t,f,"San Francisco, CA, United States",Noe Valley,Noe Valley,NaN,San Francisco,CA,94114,D.C.,"San Francisco, CA",US,United States,37.75241,-122.43169,t,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,Heating,Washer,Dryer...",NaN,$150.00,NaN,NaN,NaN,$60.00,1,$0.00,30,30,30,30,30,30,30.0,30.0,a week ago,t,0,0,0,70,10/14/19,32,0,1/3/17,1/14/18,95.0,10.0,10.0,10.0,10.0,10.0,9.0,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.95


La variable license n'a pas l'air exploitable. De plus, nous pensons qu'elle n'apporte pas grand chose à notre étude.

In [39]:
var_to_drop.append('license')

On garde seulement la colonne availability_365

In [40]:
data[['availability_30', 'availability_60', 'availability_90', 'availability_365']].head()

,availability_30,availability_60,availability_90,availability_365
0,3,8,16,85
1,5,32,62,62
2,0,0,0,0
3,30,60,90,365
4,30,60,90,365


In [41]:
var_quant.append('availability_365')
var_to_drop.extend(['availability_30', 'availability_60', 'availability_90'])

On garde juste la colonne number_of_reviews_ltm

In [42]:
var_quant.append("number_of_reviews_ltm")
var_to_drop.append("number_of_reviews")

Entre *host_listings_count* et *calculated_host_listings_count*, nous conservons *host_listings_count* car celui ci contient plus d'informations. En effet, cette variable prend aussi en compte les logements des hotes non presents dans cette liste.

In [43]:
var_to_drop.append('calculated_host_listings_count')

In [44]:
tmp = data["host_listings_count"] - data["host_total_listings_count"]

In [45]:
print(data["host_listings_count"].shape)
print(data["host_total_listings_count"].shape)
print(tmp.shape)
print(tmp.describe())

(8111,)
(8111,)
(8111,)
count    8103.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
dtype: float64


On constate que les colonnes *host_listings_count* et *host_total_listings_count*  sont identiques. On peut donc supprimer l'une des 2.  

In [46]:
var_to_drop.append("host_total_listings_count")

On garde deja "host_listings_count", donc nous pensons que nous pouvons supprimer les variables suivantes, qui apportent presque la même information.

In [47]:
var_to_drop.extend(['calculated_host_listings_count_entire_homes',
               'calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms'])

In [48]:
data['host_location'].value_counts()

San Francisco, California, United States                          6190
US                                                                 764
New York, New York, United States                                  224
Los Angeles, California, United States                             122
California, United States                                           59
San Diego, California, United States                                47
Calgary, Alberta, Canada                                            44
Penn Valley, California, United States                              35
Birmingham, Alabama, United States                                  34
San Anselmo, California, United States                              28
Denver, Colorado, United States                                     27
Tigard, Oregon, United States                                       27
Hawthorne, New Jersey, United States                                26
Victoria, British Columbia, Canada                                  26
Irvine

Il y a trop de categories differents, nous choisissons de ne pas faire de regroupements donc nous supprimons ces variables

In [49]:
var_to_drop.extend(['host_location', 'host_neighbourhood'])

# 2.3 Variables that need to be cast corectly

## 2.3.1 Simple cast

In [50]:
float_to_int = ["minimum_nights_avg_ntm", "maximum_nights_avg_ntm"]

for col in float_to_int:
    data[col] = data[col].astype('int64')
    
var_quant.extend(float_to_int)

We cannot cast these values to int because of the NAN. It must remain floats.

In [51]:
var_quant.extend(["bathrooms", "bedrooms", "beds", "host_listings_count"])

## 2.3.2 cast with missing values

In [52]:
tot = data["square_feet"].shape
nb = data["square_feet"].isna().sum()

print("Nombre de valeurs ayant une superficie : "
      + str((tot - nb)[0]) )

print("Cela représente " + str(( (100*(tot - nb))/nb )[0]) + "% de nos données")

Nombre de valeurs ayant une superficie : 124
Cela représente 1.5525228496306498% de nos données


Il va falloir réflechir à comment traiter ces valeurs manquantes (dans le notebook suivant)

In [53]:
var_to_drop.append("square_feet")

## 2.3.3 cast price to int

On convertit le prix en entier.

In [54]:
price_to_int = ['price', 'extra_people', 'security_deposit', 'cleaning_fee']

In [55]:
def money_to_int(x):
    if x != x:        # test si x == NAN
        return x
    else:
        return int( x[1 : (len(x)-4) ].replace(',', '') )

In [56]:
for col in price_to_int:
    data[col] = data[col].map( lambda x: money_to_int(x) )

In [57]:
var_quant.extend(price_to_int)

In [58]:
def per_to_float(x):
    if x != x:        # test si x == NAN
        return x
    else:
        return  x[0 : -1 ]

In [59]:
data['host_response_rate'] = data['host_response_rate'].map( lambda x: per_to_float(x) )

In [60]:
var_quant.append('host_response_rate')

Est ce qu'on garde les autres prix? Je suis d'avis de les enlever

In [61]:
var_to_drop.extend(['monthly_price', 'weekly_price'])

## 2.3.4 dates

In [62]:
var_date.extend(['first_review', 'last_review', 'host_since'])

In [63]:
for col in var_date:
    data[col] = pd.to_datetime(data[col])

In [64]:
data[var_date].head()
#data[var_date].dtypes

,first_review,last_review,host_since
0,2009-07-23,2019-09-21,2008-07-31
1,2009-07-16,2019-09-30,2008-12-08
2,2009-05-03,2017-08-06,2009-03-02
3,2009-08-31,2019-09-10,2009-06-17
4,2014-09-08,2018-09-12,2009-06-17


In [65]:
recent_date = data[var_date].max()

In [66]:
for col in var_date:
    data[col] = recent_date[col] - data[col]
    data[col] = data[col].dt.days

# 2.4 Categorical variables

In [67]:
data['city'].value_counts()

San Francisco                  8059
Daly City                        34
San Francisco                     3
Brisbane                          1
Noe Valley - San Francisco        1
San Jose                          1
San Francisco, Hayes Valley       1
旧金山                               1
Name: city, dtype: int64

In [68]:
data['smart_location'].value_counts()

San Francisco, CA                  8050
Daly City, CA                        34
CA                                    9
San Francisco                         6
San Francisco , CA                    3
San Francisco, Ca                     3
San Jose, CA                          1
Brisbane, CA                          1
旧金山, CA                               1
US,                                   1
San Francisco, Hayes Valley, CA       1
Noe Valley - San Francisco, CA        1
Name: smart_location, dtype: int64

In [69]:
var_cat.append('city')
var_to_drop.append("smart_location")

Est ce qu'on a vraiment besoin des 2 variables ?  
On peut garder la variable *city* pour l'instant. On peut éventuellement retirer la ligne écrite en chinois.

In [70]:
data['host_response_time'].value_counts()

within an hour        5142
within a few hours    1341
within a day           625
a few days or more      76
Name: host_response_time, dtype: int64

In [71]:
var_cat.append('host_response_time')

In [72]:
cast_to_cat = ['property_type', 'room_type', 'bed_type', 'cancellation_policy']

In [73]:
def print_cat(data, l):
    for col in l:
        print(data[col].value_counts())
        print("")

In [74]:
#print_cat(data, cast_to_cat)

In [75]:
var_cat.extend(cast_to_cat)

Il faudrait réduire le nombre de catégories de *property_type*.

## Colonnes *neighbourhood* et *neighbourhood_cleansed*

In [76]:
data[['neighbourhood', 'neighbourhood_cleansed']].head()

,neighbourhood,neighbourhood_cleansed
0,Duboce Triangle,Western Addition
1,Inner Sunset,Inner Sunset
2,Bernal Heights,Bernal Heights
3,Cole Valley,Haight Ashbury
4,Cole Valley,Haight Ashbury


La variable *neighbourhood_cleansed* a l'air d'être une version prétraitée de *neighbourhood*. NOus pouvons donc seulement garder *neighbourhood_cleansed*.  
Est ce qu'on traite cette colonne comme du texte ou comme une variable catégorique?

In [77]:
var_space.append('neighbourhood_cleansed')

In [78]:
var_to_drop.append('neighbourhood')

# 2.5 Boolean variables

In [79]:
var_bool = ['requires_license', 'instant_bookable',
                'require_guest_profile_picture', 'require_guest_phone_verification',
                'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified']

In [80]:
def print_bool(data, l):
    for col in l:
        print(data[col].value_counts())
        print("")

In [81]:
#print_bool(data, var_bool)

In [82]:
d = {'t': True, 'f': False}

In [83]:
for col in var_bool:
    data[col] = data[col].map(d)

## Colonne *is_location_exact*

La variable is_location_exact ne semble pas être pertinente pour notre analyse. On la supprime.

In [84]:
var_to_drop.append('is_location_exact')

## Colonnes *amenities* et *host_verifications*

Pour chaque logement, nous disosons d'une liste. Il y a trop de valeurs différentes pour en faire des categories. Nous allons donc faire un *count* de ces valeurs.

In [85]:
data['amenities'].head()

0    {TV,"Cable TV",Internet,Wifi,Kitchen,"Pets liv...
1    {Internet,Wifi,Kitchen,Breakfast,"Free street ...
2    {Internet,Wifi,Kitchen,Heating,"Family/kid fri...
3    {TV,Internet,Wifi,Kitchen,"Free street parking...
4    {TV,Internet,Wifi,Kitchen,"Free street parking...
Name: amenities, dtype: object

In [86]:
data['host_verifications'].head()

0     ['email', 'phone', 'facebook', 'reviews', 'kba']
1    ['email', 'phone', 'reviews', 'jumio', 'govern...
2    ['email', 'phone', 'reviews', 'kba', 'work_ema...
3    ['email', 'phone', 'reviews', 'jumio', 'govern...
4    ['email', 'phone', 'reviews', 'jumio', 'govern...
Name: host_verifications, dtype: object

In [87]:
amenities_count = []
host_verif_count = []

for row in data.itertuples(index=False):    
    amenities_count.append(len(row.amenities[1:-1].replace('"','').split(",")))
    host_verif_count.append(len(row.host_verifications[1:-1].replace('"','').split(",")))

In [88]:
data['amenities'] = amenities_count
data['host_verifications'] = host_verif_count

In [89]:
var_quant.extend(["amenities", 'host_verifications'])

# 2.6 Variables with outliers

## *minimum_nights* et *maximum_nights*

In [90]:
def add_row_to_drop(data, variable, condition):
    l = []
    
    for value in condition:
        l.append(data[data[variable] == value].index[0])
    
    return l

In [91]:
condition = [100000000, 1125, 1000]
row_to_drop.extend(add_row_to_drop(data, "minimum_nights", condition))

In [92]:
condition = [100000, 10000, 9999]
row_to_drop.extend(add_row_to_drop(data, "maximum_nights", condition))

In [93]:
var_quant.extend(["maximum_nights", "minimum_nights"])

On constate des "paliers" pour les valeurs 30, 60, 90 et 365. Est ce qu'il y a un rapport avec les variables "availibility_xx". Si c'est le cas, on risque d'avoir un problème de multicolinéarité et il faudra surement faire une selection parmi ces variables.

## *minimum_nights_avg_ntm* and *maximum_nights_avg_ntm*

D'après cette source (https://eprints.ucm.es/57354/1/TFM_Rentalbility_PriscillaToscano_VF_160919.pdf)  
minimum_nights_avg_ntm = Minimum Nights in Avg from last Twelve Month, ce qui me parait etrange

In [94]:
condition = [100000000, 1125, 1000]
row_to_drop.extend(add_row_to_drop(data, "minimum_nights_avg_ntm", condition))

In [95]:
condition = [2147483647, 100000, 10000, 9999, 2000, 1825]
row_to_drop.extend(add_row_to_drop(data, "maximum_nights_avg_ntm", condition))

## *city*

In [96]:
row_to_drop.append(data[data["city"] == "旧金山"].index[0])

## 3.2 Hosts

In [97]:
host_close = []

for row in data.itertuples(index = False): 
    host_close.append( row.host_location == 'San Francisco, California, United States' )

In [98]:
#host_close

In [99]:
data["host_is_close"] = host_close
var_bool.append('host_is_close')

# 4. Check if all variables have been treated one time*

In [100]:
control = var_quant + var_cat + var_bool + var_text + var_date + var_to_drop + var_TODO + var_cible + var_space

In [101]:
print(len(control))
print(len(set(control)))
print(len(data.columns))

107
107
107


In [102]:
len(var_cible + var_quant + var_cat + var_bool + var_space + var_date + var_text)

51

In [103]:
len(var_to_drop)

56

# 5. Save the cleanned data

In [104]:
row_to_drop.extend(data[data['review_scores_rating'].isna()].index)

In [105]:
data_clean = data[var_cible + var_quant + var_cat + var_bool + var_space + var_date + var_text]
data_clean = data_clean.drop(row_to_drop)

In [106]:
data_clean.head()

,review_scores_rating,accommodates,guests_included,availability_365,number_of_reviews_ltm,minimum_nights_avg_ntm,maximum_nights_avg_ntm,bathrooms,bedrooms,beds,host_listings_count,price,extra_people,security_deposit,cleaning_fee,host_response_rate,amenities,host_verifications,maximum_nights,minimum_nights,city,host_response_time,property_type,room_type,bed_type,cancellation_policy,requires_license,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,host_is_superhost,host_has_profile_pic,host_identity_verified,host_is_close,latitude,longitude,neighbourhood_cleansed,first_review,last_review,host_since,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,host_about
0,97.0,3,2,85,52,1,30,1.0,1.0,2.0,1.0,170,25,100.0,100.0,100,32,5,30,1,San Francisco,within a day,Apartment,Entire home/apt,Real Bed,moderate,True,False,False,False,True,True,True,True,37.76931,-122.43386,Western Addition,3734.0,22.0,4089.0,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under const...,"Newly remodeled, modern, and bright garden uni...",New update: the house next door is under const...,*Quiet cul de sac in friendly neighborhood *St...,Due to the fact that we have children and a do...,*Public Transportation is 1/2 block away. *Ce...,*Full access to patio and backyard (shared wit...,A family of 4 lives upstairs with their dog. N...,* No Pets - even visiting guests for a short t...,We are a family with 2 boys born in 2009 and 2...
1,94.0,2,2,62,36,1,5,1.0,1.0,1.0,2.0,99,20,0.0,10.0,100,30,5,5,1,San Francisco,within an hour,House,Private room,Real Bed,strict_14_with_grace_period,True,False,False,False,True,True,True,True,37.75402,-122.45805,Inner Sunset,3741.0,13.0,3959.0,Charming room for two,Your own private room plus access to a shared ...,This room can fit two people. Nobody else will...,Your own private room plus access to a shared ...,"This is a quiet, safe neighborhood on a substa...",House Rule footnotes: 1.\tI don’t allow check ...,Public transit service to my house is outstand...,"During the hours of 7:00 to 22:00, your access...","I live at the house in my own bedroom, and I w...",Airbnb has no default house rules that hosts c...,I am a retired software developer now raising ...
2,98.0,5,2,0,0,30,60,1.0,2.0,3.0,2.0,235,0,NaN,100.0,80,17,5,60,30,San Francisco,within a day,Apartment,Entire home/apt,Real Bed,strict_14_with_grace_period,True,False,False,False,False,True,True,True,37.74511,-122.42102,Bernal Heights,3815.0,798.0,3875.0,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,I love how our neighborhood feels quiet but is...,All the furniture in the house was handmade so...,The train is two blocks away and you can stop ...,"Our deck, garden, gourmet kitchen and extensiv...",NaN,"Please respect the house, the art work, the fu...",Philip: English transplant to the Bay Area and...
3,86.0,2,1,365,1,32,60,4.0,1.0,1.0,10.0,65,12,200.0,50.0,86,18,5,60,32,San Francisco,within an hour,Apartment,Private room,Real Bed,strict_14_with_grace_period,True,False,False,False,True,True,True,True,37.76669,-122.45250,Haight Ashbury,3695.0,33.0,3768.0,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,"Settle down, S.F. resident, student, hospital,...",Nice and good public transportation. 7 minute...,"Shopping old town, restaurants, McDonald, Whol...",Wi-Fi signal in common areas. Large eat in k...,N Juda Muni and bus stop. Street parking.,NaN,NaN,"No party, No smoking, not for any kinds of smo...",7 minutes walk to UCSF hospital & school campu...
4,93.0,2,1,365,0,32,90,4.0,1.0,1.0,10.0,65,12,200.0,50.0,86,16,5,90,32,San Francisco,within an hour,Apartment,Private room,Real Bed,strict_14_with_grace_period,True,False,False,False,True,True,True,True,37.76487,-122.45183,Haight Ashbury,1861.0,396.0,3768.0,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice a

In [107]:
row_to_drop[-1]

8110

In [108]:
meta_data = {}
meta_data['y'] = var_cible[0]
meta_data['quant'] = var_quant
meta_data['cat'] = var_cat
meta_data['bool'] = var_bool
meta_data['space'] = var_space
meta_data['date'] = var_date
meta_data['text'] = var_text

In [109]:
with open('../data/meta_data.json', 'w') as outfile:
    json.dump(meta_data, outfile)
    
data_clean.to_csv ('../data/clean.csv',index=False, header=True)